## Imports

In [ ]:
%load_ext lab_black

import h5py
import os
import numpy as np
from typing import Dict, List, Optional, Tuple

from dataclasses import dataclass
from tqdm.auto import tqdm
from scipy.signal import savgol_filter
from scipy.interpolate import interp2d
from functools import lru_cache
import lmfit as lm


import plotly.graph_objects as go
import plotly.colors as pc
import matplotlib.pyplot as plt


import sys

sys.path.append(r"C:\Users\atully\Code\GitHub\ARPES Code\arpes-code-python")
from arpes_functions import (
    fitting_functions,
    analysis_functions,
    plotting_functions,
    HDF5_loader,
    misc_functions,
    filter_functions,
    tr_functions,
    loading_functions,
    kw_data_loader,
    cnn,
)

colors = pc.qualitative.D3
colors_seq = pc.sequential.dense
angstrom = "\u212B"
Theta = "\u0398"

# Load Data

In [ ]:
# ## Convert Data ##

# ddir = r"E:\atully\arpes_data\2023_February\6eV"

# # STEP 1 ##
# # Convert ibw to hdf5
# fn = "Img29_g_kw.ibw"
# HDF5_loader.ibw_to_hdf5(ddir, fn, export=True)

# # # Check conversion worked
# # data, theta, energy = HDF5_loader.load_hdf5(
# #     ddir, "Img7_Lamp_g_kw.h5"
# # )  # load data from hdf5
# # data.shape, theta.shape, energy.shape

In [ ]:
ddir = r"E:\atully\arpes_data\2023_February\6eV"

cone_data, cone_kx, cone_energy = HDF5_loader.load_hdf5(ddir, "Img29_g_kw.h5")

In [ ]:
xaxis_title = f"k<sub>x</sub> ({angstrom}<sup>-1</sup>)"
yaxis_title = f"E - E<sub>HOMO</sub> (eV)"

# Set Up Data

In [ ]:
## HOMO is at 2.05 eV below EF, based on fits from this data averaged with fits from tr-ARPES results ##

EF_400 = 1.91  # in kinetic energy, slit 400
EF_700 = 1.94  # in kinetic energy, slit 700

homo = -2.05

homo_400 = homo + EF_400

energy = cone_energy - homo_400

In [ ]:
## Limit Dataset and FFT ##

data, x, y = cone_data, kx, energy

xlim = None
ylim = None
# ylim = (energy[57], energy[1007])  # get rid of zero padding on datasets
ylim = (2.052, 2.66)
xlim = (-0.22, 0.21)
xlim = (-0.15, 0.15)

f_data = filter_functions.fft2d_mask(data, plot=False)

x_plot, y_plot, data_plot = analysis_functions.limit_dataset(
    x,
    y,
    f_data,
    xlim=xlim,
    ylim=ylim,
)

# d_avg = analysis_functions.norm_data(d_avg)

In [ ]:
## Plot ##

x_plot, y_plot, data_plot = x_plot, y_plot, data_plot

fig = tr_functions.thesis_fig(
    title="Mystery Feature",
    xaxis_title=xaxis_title,
    yaxis_title=yaxis_title,
    equiv_axes=False,
    height=600,
    width=600,
)

fig.add_trace(
    go.Heatmap(
        x=x_plot,  # fix incorrect traces by, first, plotting without x-axis
        y=y_plot,  # fix incorrect traces by, first, plotting without y-axis
        # z=data_plot,
        z=analysis_functions.norm_data(data_plot),
        coloraxis="coloraxis",
    )
)

fig.update_coloraxes(cmin=0, cmax=0.15)

fig.show()